In [1]:
import pandas as pd
import numpy as np
import os
import math
from sklearn.cluster import KMeans
%config IPCompleter.greedy=True

In [2]:
# Data Files
OUTPUT_DIR = 'outputs/'
INPUT_DIR = 'data/'

HOD_same = 'HumanObserved-Dataset/same_pairs.csv'
HOD_diff = 'HumanObserved-Dataset/diffn_pairs.csv'
HOD_features = 'HumanObserved-Dataset/HumanObserved-Features-Data.csv'

GSC_same = 'GSC-Dataset/same_pairs.csv'
GSC_diff = 'GSC-Dataset/same_pairs.csv'
GSC_features = 'GSC-Dataset/GSC-Features.csv'

In [3]:
# Linear Regression Variables
TrainingPercent = 80 # Data Split for Training Data
ValidationPercent = 10 # Data Split for Validation Data
TestPercent = 10 # Data Split for Testing Data


M = 10
C_Lambda = 0.03
learningRate = 0.01


In [4]:
def _remove_zero_cols(data):
    """Returns ndarray with no zero columns
    """
    data = data.transpose()
    data = data[~np.all(data == 0, axis=1)]
    data = data.transpose()
    return data


def _append_dataframes(same_data, diff_data):
    """Appends the dataframes one after another.
    """
    appended_data = pd.concat([same_data, diff_data])
    appended_data = appended_data.sample(frac=1)
    return(appended_data)


In [5]:
def _concat_dataframes(same_data, diff_data, features_data):
    """
    """
    # Combine the diff and concat data
    appended_data = _append_dataframes(same_data, diff_data)
    # Remove columns if exist
    if 'Unnamed: 0' in features_data.columns.values:
        features_data = features_data.drop(columns=['Unnamed: 0'])

    # Merge the datatables, same and diff
    merged_data = pd.merge(appended_data, features_data,
                           left_on='img_id_A', right_on='img_id', how='left')
    merged_data = pd.merge(merged_data, features_data, left_on='img_id_B',
                           right_on='img_id', how='left', suffixes=('_a', '_b'))
    merged_data = merged_data.drop(columns=['img_id_a', 'img_id_b'])
    raw_data = merged_data.loc[:, 'f1_a':].values
    raw_data = _remove_zero_cols(raw_data)
    raw_target = merged_data['target'].values
    return(raw_data, raw_target)


def _subtract_dataframes(same_data, diff_data, features_data):
    """Subtract img_id_a columns with img_id_b_columns
    """
    appended_data = _append_dataframes(same_data, diff_data)
    temp1 = pd.merge(appended_data, features_data,
                     left_on='img_id_A', right_on='img_id', how='left')
    temp2 = pd.merge(appended_data, features_data,
                     left_on='img_id_B', right_on='img_id', how='left')
    raw_data = abs(temp1.loc[:, 'f1':] - temp2.loc[:, 'f1':]).values

    raw_data = _remove_zero_cols(raw_data)
    raw_target = appended_data['target'].values

    return(raw_data, raw_target)


In [6]:
def get_data_features(category='hod', operation='concat'):
    """Returns concat/subtracted feature values
    Merge the same and diff datasets, and process the data columns
    """
    category_dict = {
        'hod': [HOD_same, HOD_diff, HOD_features],
        'gsc': [GSC_same, GSC_diff, GSC_features],
    }
    same, diff, features = category_dict[category]
    same_data = pd.read_csv(os.path.join(INPUT_DIR, same), sep=',', nrows=1000)
    diff_data = pd.read_csv(os.path.join(
        INPUT_DIR, diff),  sep=',', nrows=1000)
    features_data = pd.read_csv(os.path.join(INPUT_DIR, features), sep=',')

    if operation == 'concat':
        raw_data, raw_target = _concat_dataframes(
            same_data, diff_data, features_data)

    if operation == 'subtract':
        raw_data, raw_target = _subtract_dataframes(
            same_data, diff_data, features_data)

    return(raw_data, raw_target)


In [18]:
# data1_d, data1_t = get_data_features('hod', operation='concat')
# print(data1_d.shape, data1_t.shape)
data1_d, data1_t = get_data_features('hod', operation='subtract')
print(data1_d.shape, data1_t.shape)
# data1_d, data1_t = get_data_features('gsc', operation='concat')
# print(data1_d.shape, data1_t.shape)
# data4_d, data4_t = get_data_features('gsc', operation='subtract')
# print(data4_d.shape, data4_t.shape)

(1791, 9) (1791,)


In [19]:
training_ratio = math.floor(data1_d.shape[0]*.8)
val_test_ratio = math.floor(data1_d.shape[0]*.9)

training_data, training_target = data1_d[:training_ratio], data1_t[:training_ratio]
validation_data, validation_target  = data1_d[training_ratio:val_test_ratio], data1_t[training_ratio:val_test_ratio]
testing_data, testing_target = data1_d[val_test_ratio:], data1_t[val_test_ratio:]
data1_d, training_data, testing_data, validation_data = data1_d.transpose(), training_data.transpose(), testing_data.transpose(), validation_data.transpose()
print(training_data.shape, testing_data.shape, validation_data.shape)
print(training_target.shape, testing_target.shape, validation_target.shape)
# 

(9, 1432) (9, 180) (9, 179)
(1432,) (180,) (179,)


In [20]:
# num_observations = 5000

# x1 = np.random.multivariate_normal([0, 0], [[1, .75],[.75, 1]], num_observations)
# x2 = np.random.multivariate_normal([1, 4], [[1, .75],[.75, 1]], num_observations)
# simulated_separableish_features = np.vstack((x1, x2)).astype(np.float32)
# simulated_labels = np.hstack((np.zeros(num_observations),
#                               np.ones(num_observations)))
# print(simulated_separableish_features.shape)
# print(simulated_labels.shape)

In [21]:
def sigmoid(var):
    return 1 / (1 + np.exp(-var))


In [22]:
# def log_likelihood(features, target, weights):
#     scores = np.dot(features, weights)
#     ll = np.sum( target*scores - np.log(1 + np.exp(scores)) )
#     return ll

In [23]:
# def logistic_regression(features, target, num_steps, learning_rate, add_intercept = False):
#     if add_intercept:
#         intercept = np.ones((features.shape[0], 1))
#         features = np.hstack((intercept, features))
        
#     weights = np.zeros(features.shape[1])
    
#     for step in range(num_steps):
#         scores = np.dot(features, weights)
#         predictions = sigmoid(scores)

#         # Update weights with gradient
#         output_error_signal = target - predictions
#         gradient = np.dot(features.T, output_error_signal)
#         weights += learning_rate * gradient
        
#         # Print log-likelihood every so often
#         if step % 10000 == 0:
#             print(log_likelihood(features, target, weights))
        
#     return weights

In [24]:
# weights = logistic_regression(training_data.transpose(), training_target,
#                      num_steps = 300000, learning_rate = 5e-5, add_intercept=True)

In [25]:
def gradient(w, x, y):
    w = np.matrix(w)
    x = np.matrix(x)
    y = np.matrix(y)
    
    par = int(w.ravel().shape[1]) #Number of features
    grad = np.zeros(par)
    
    error = sigmoid(x * w.T) - y
    for i in range(0,par):
        term = np.multiply(error, x[:,i])
        grad[i] = np.sum(term)/len(x)
        
    return grad

In [30]:
W_Now = np.random.rand(training_data.shape[0])
# print(training_data.shape[0])
learningRate = 0.001
bias = 2

9


In [31]:
la = 2
for i in range(0,len(training_data[0])):
    #print("Loss for " +str(i)+ "epoch: " +str(cost(W_Now, TrainingData.T, TrainingTarget)))
    delta_W = gradient(W_Now, training_data.T, training_target)
    La_delta_W  = np.dot(la, W_Now)
    delta_W_Reg = np.add(delta_W, La_delta_W)
    W_Next = W_Now - (np.dot(learningRate, delta_W_Reg))
    W_Now = W_Next
    print(i, 'New update :', W_Now)

0 New update : [-0.42549819  0.14769591  0.29025207 -0.66904142  0.43013569  0.35183499
  0.16613195 -0.52071187  0.50867834]
1 New update : [-0.16932654  0.31351636  0.30197403 -0.09187033  0.44181542  0.42560377
  0.28365513 -0.11170866  0.52598766]
2 New update : [-0.36852002 -0.06965277  0.25018413 -0.40175555  0.28240153  0.26116469
  0.02675892 -0.40122355  0.43472148]
3 New update : [-0.11845674  0.16974523  0.26628848  0.09077066  0.31224037  0.33952099
  0.1711097  -0.01118326  0.45523395]
4 New update : [-0.36833601 -0.20216558  0.21335167 -0.38211072  0.15698795  0.16372962
 -0.09067924 -0.39251962  0.36210982]
5 New update : [-0.06465824  0.14230062  0.2435352   0.17965893  0.23313378  0.28430651
  0.12605426  0.07115324  0.40528528]
6 New update : [-0.36664942 -0.26093605  0.18782281 -0.40556271  0.07633709  0.09421883
 -0.15588213 -0.39710767  0.30823311]
7 New update : [-0.03137198  0.13282198  0.22581105  0.20800475  0.17821969  0.240212
  0.09988317  0.11189263  0.3651

69 New update : [0.10400185 0.14435852 0.05784311 0.23539579 0.03400056 0.04067144
 0.08546857 0.17959571 0.03576986]
70 New update : [-0.24078437 -0.27508212  0.00872484 -0.40845972 -0.09664638 -0.13341848
 -0.1962542  -0.3491991  -0.04395856]
71 New update : [0.1042494  0.14437219 0.0559649  0.23542319 0.03446555 0.04088243
 0.08573248 0.17956191 0.03366296]
72 New update : [-0.24055492 -0.2750733   0.00690184 -0.40844422 -0.09620025 -0.13320728
 -0.19600607 -0.34923921 -0.04597247]
73 New update : [0.10447551 0.14438471 0.05417702 0.23544829 0.03490209 0.04109582
 0.08597836 0.1795287  0.03171143]
74 New update : [-0.24034491 -0.27506492  0.00516654 -0.4084297  -0.09578152 -0.13299512
 -0.19577455 -0.34927768 -0.04783792]
75 New update : [0.10468239 0.14439631 0.05247503 0.23547132 0.03531159 0.04130894
 0.08620757 0.17949632 0.02990359]
76 New update : [-0.2401524  -0.27505687  0.00351462 -0.40841607 -0.0953888  -0.13278436
 -0.19555839 -0.34931444 -0.04956603]
77 New update : [0.1

137 New update : [0.10695187 0.1446081  0.02604925 0.23574633 0.0405853  0.04494269
 0.08912053 0.17901051 0.00916304]
138 New update : [-0.23801183 -0.27486653 -0.02212678 -0.40822245 -0.09032238 -0.12925606
 -0.19278152 -0.34981122 -0.06939214]
139 New update : [0.10696999 0.14461221 0.02569662 0.23574912 0.04063771 0.04498425
 0.08914973 0.17900573 0.00900231]
140 New update : [-0.23799444 -0.27486246 -0.02246881 -0.40822011 -0.09027179 -0.12921591
 -0.19275332 -0.34981567 -0.06954576]
141 New update : [0.10698687 0.14461619 0.02536098 0.23575176 0.040687   0.04502334
 0.08917721 0.17900127 0.00885322]
142 New update : [-0.23797822 -0.27485853 -0.02279436 -0.40821788 -0.09022421 -0.12917813
 -0.19272677 -0.34981982 -0.06968826]
143 New update : [0.1070026  0.14462002 0.02504152 0.23575424 0.04073337 0.04506013
 0.08920307 0.1789971  0.00871492]
144 New update : [-0.23796309 -0.27485474 -0.02310423 -0.40821577 -0.09017945 -0.12914258
 -0.19270178 -0.34982369 -0.06982044]
145 New upda

206 New update : [-0.23777483 -0.27478797 -0.02789496 -0.40818446 -0.08956444 -0.1286571
 -0.19235567 -0.34987064 -0.07135357]
207 New update : [0.10719908 0.14468835 0.02003627 0.23579036 0.04137631 0.04556719
 0.08956362 0.17894445 0.00709805]
208 New update : [-0.23777306 -0.27478695 -0.02795845 -0.40818407 -0.08955752 -0.12865173
 -0.19235173 -0.34987106 -0.0713657 ]
209 New update : [0.10720077 0.14468933 0.01997397 0.23579077 0.04138306 0.04557241
 0.08956743 0.178944   0.00708625]
210 New update : [-0.2377714  -0.27478598 -0.02801886 -0.4081837  -0.08955098 -0.12864665
 -0.19234801 -0.34987145 -0.07137698]
211 New update : [0.10720236 0.14469026 0.01991468 0.23579115 0.04138943 0.04557735
 0.08957104 0.17894357 0.0070753 ]
212 New update : [-0.23776984 -0.27478506 -0.02807635 -0.40818334 -0.08954479 -0.12864186
 -0.19234448 -0.34987181 -0.07138745]
213 New update : [0.10720385 0.14469115 0.01985827 0.23579152 0.04139546 0.04558202
 0.08957445 0.17894316 0.00706512]
214 New updat

275 New update : [0.10722352 0.14470551 0.01898824 0.235797   0.04148289 0.04564908
 0.08962409 0.17893784 0.00694518]
276 New update : [-0.237749   -0.27476997 -0.02897469 -0.40817795 -0.089454   -0.12857211
 -0.19229251 -0.34987649 -0.07151176]
277 New update : [0.10722372 0.14470571 0.01897674 0.23579707 0.04148396 0.04564989
 0.0896247  0.17893778 0.00694419]
278 New update : [-0.23774879 -0.27476977 -0.02898584 -0.40817788 -0.08945297 -0.12857132
 -0.19229191 -0.34987653 -0.07151271]
279 New update : [0.10722392 0.1447059  0.01896581 0.23579713 0.04148497 0.04565066
 0.08962527 0.17893773 0.00694327]
280 New update : [-0.2377486  -0.27476958 -0.02899645 -0.40817782 -0.08945198 -0.12857058
 -0.19229134 -0.34987657 -0.07151359]
281 New update : [0.1072241  0.14470608 0.0189554  0.23579719 0.04148593 0.04565138
 0.08962582 0.17893768 0.00694241]
282 New update : [-0.23774841 -0.27476941 -0.02900654 -0.40817776 -0.08945105 -0.12856987
 -0.19229081 -0.34987661 -0.07151441]
283 New upda

...

1166 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1167 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1168 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1169 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1170 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1171 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1172 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1173 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1174

1232 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1233 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1234 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1235 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1236 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1237 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1238 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1239 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1240

1298 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1299 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1300 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1301 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1302 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1303 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1304 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1305 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1306

1364 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1365 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1366 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1367 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1368 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1369 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1370 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1371 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]
1372

1430 New update : [-0.23774524 -0.27476584 -0.02920423 -0.40817663 -0.08943347 -0.12855663
 -0.19228064 -0.34987724 -0.0715258 ]
1431 New update : [0.10722726 0.14470969 0.01875151 0.2357984  0.04150398 0.04566495
 0.08963614 0.17893682 0.00693048]


In [28]:
Y_Test_rounded = np.rint(sigmoid(np.matmul(testing_data.T, W_Now)))
diff = np.subtract(Y_Test_rounded, testing_target)
non_zeros = np.count_nonzero(diff)
accuracy = ((len(diff) - non_zeros)/len(diff))*100
print(accuracy)


48.333333333333336
